In [2]:
import math
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Legend,Plot,GraphRenderer, Oval, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool,BoxZoomTool, Rect, WheelZoomTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes,from_networkx
from bokeh.palettes import Spectral4, Spectral8, Blues8, Greens8, magma,OrRd,Oranges, Viridis256, Oranges8
from bokeh.plotting import ColumnDataSource
from bokeh.models import ColumnDataSource, LabelSet, ColumnData,StaticLayoutProvider,GraphRenderer
from bokeh.layouts import gridplot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
import csv

In [4]:
airlines = pd.read_csv('flight-delays/airlines.csv')
airports = pd.read_csv('flight-delays/airports.csv')

In [3]:
flights = pd.read_csv('flight-delays/flights.csv')

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
flights = flights[(flights['MONTH']==1) & ((flights['DAY']==1)|(flights['DAY']==2)|(flights['DAY']==3)|(flights['DAY']==4)|(flights['DAY']==5))]
new = flights.merge(airports, how='left', left_on='ORIGIN_AIRPORT', right_on = 'IATA_CODE')
new = new.rename(columns = {"LATITUDE": "ORIGIN_LAT", "LONGITUDE":"ORIGIN_LON"})
new2 = new.merge(airports, how='left', left_on='DESTINATION_AIRPORT', right_on = 'IATA_CODE')
new2 = new2.rename(columns = {"LATITUDE": "DESTINATION_LAT", "LONGITUDE":"DESTINATION_LON"})

In [19]:
len(new2)

79025

In [57]:
airports.dtypes

IATA_CODE          object
AIRPORT            object
CITY               object
STATE              object
COUNTRY            object
LATITUDE          float64
LONGITUDE         float64
HUB_CARRIERS       object
FOCUS_CARRIERS     object
dtype: object

In [6]:
airports['HUB_CARRIERS'] = airports['HUB_CARRIERS'].astype(str)

In [7]:
ls = []
for i in airports['HUB_CARRIERS']:
    ls.append(len(i.split()))

In [8]:
airports['HUB_COUNTS'] = ls

In [9]:
airports.head()

Unnamed: 0 IATA_CODE                              AIRPORT         CITY  \
0           0       ABE  Lehigh Valley International Airport    Allentown   
1           1       ABI             Abilene Regional Airport      Abilene   
2           2       ABQ    Albuquerque International Sunport  Albuquerque   
3           3       ABR            Aberdeen Regional Airport     Aberdeen   
4           4       ABY   Southwest Georgia Regional Airport       Albany   

  STATE COUNTRY  LATITUDE  LONGITUDE HUB_CARRIERS FOCUS_CARRIERS  HUB_COUNTS  \
0    PA     USA  40.65236  -75.44040         None           None           1   
1    TX     USA  32.41132  -99.68190         None           None           1   
2    NM     USA  35.04022 -106.60919         None           None           1   
3    SD     USA  45.44906  -98.42183         None           None           1   
4    GA     USA  31.53552  -84.19447         None           None           1   

   FOCUS_COUNTS  
0             0  
1             0  
2             0  
3             0  
4             0

In [61]:
airports['FOCUS_CARRIERS'] = airports['FOCUS_CARRIERS'].astype(str)

In [62]:
ls = []
for i in airports['FOCUS_CARRIERS']:
    ls.append(len(i.split()))

In [63]:
airports['FOCUS_COUNTS'] = ls

In [64]:
airports.to_csv('airports.csv')

In [23]:
flights.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

In [15]:
flights['AIRLINE'].unique()

array(['AS', 'AA', 'US', 'DL', 'NK', 'UA', 'HA', 'B6', 'OO', 'EV', 'MQ',
       'F9', 'WN', 'VX'], dtype=object)

In [16]:
lst = []
for i in flights['AIRLINE']:
    if i == "AS":
        lst.append("Alaska")
    elif i == "AA":
        lst.append("American")
    elif i == "US":
        lst.append("USAirways")
    elif i == "DL":
        lst.append("Delta")
    elif i == "NK":
        lst.append("Spirit")
    elif i == "UA":
        lst.append("United")
    elif i == "HA":
        lst.append("Hawaiian")
    elif i == "B6":
        lst.append("JetBlue")
    elif i == "OO":
        lst.append("SkyWest")
    elif i == "EV":
        lst.append("ExpressJet")
    elif i == "MQ":
        lst.append("AmericanEagle")
    elif i == "F9":
        lst.append("Frontier")
    elif i == "WN":
        lst.append("SouthWest")
    elif i == "VX":
        lst.append("VirginAmerica")
    else:
        lst.append('Unkown')

In [19]:
flights['AIRLINE_NAME'] = lst

In [26]:
flights['TOTAL_DELAY'] = flights['ARRIVAL_DELAY']+flights['DEPARTURE_DELAY']

In [27]:
flights['TOTAL_TAXI'] = flights['TAXI_IN'] + flights['TAXI_OUT']

In [28]:
flights.head(5)

YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   
3  2015      1    1            4      AA            258      N3HYAA   
4  2015      1    1            4      AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE     ...      \
0            ANC                 SEA                    5     ...       
1            LAX                 PBI                   10     ...       
2            SFO                 CLT                   20     ...       
3            LAX                 MIA                   20     ...       
4            SEA                 ANC                   25     ...       

   CANCELLED  CANCELLATION_REASON  AIR_SYSTEM_DELAY  SECURITY_DELAY  \
0          0                  NaN               NaN             NaN   
1          0                  NaN               NaN             NaN   
2          0                  NaN               NaN             NaN   
3          0                  NaN               NaN             NaN   
4          0                  NaN               NaN             NaN   

   AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  WEATHER_DELAY  AIRLINE_NAME  \
0            NaN                  NaN            NaN        Alaska   
1            NaN                  NaN            NaN      American   
2            NaN                  NaN            NaN     USAirways   
3            NaN                  NaN            NaN      American   
4            NaN                  NaN            NaN        Alaska   

   TOTAL_DELAY  TOTAL_TAXI  
0        -33.0        25.0  
1        -17.0        16.0  
2          3.0        27.0  
3        -14.0        23.0  
4        -22.0        16.0  

[5 rows x 34 columns]

In [29]:
lst = []
for i in flights['TOTAL_DELAY']:
    if i < 0:
        lst.append('Early')
    elif i == 0:
        lst.append('On Time')
    else:
        lst.append('Late')

In [30]:
flights['EARLY_FLAG'] = lst

In [31]:
flights.to_csv('flight-delays/dataset_flights.csv')

In [37]:
airports.isnull().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

In [36]:
airports[airports['IATA_CODE']=='ATL']

IATA_CODE                                           AIRPORT     CITY STATE  \
20       ATL  Hartsfield-Jackson Atlanta International Airport  Atlanta    GA   

   COUNTRY  LATITUDE  LONGITUDE  
20     USA  33.64044  -84.42694

In [38]:
len(airports)

322

In [40]:
flights['AIRLINE'].unique()

array(['AS', 'AA', 'US', 'DL', 'NK', 'UA', 'HA', 'B6', 'OO', 'EV', 'MQ',
       'F9', 'WN', 'VX'], dtype=object)

In [ ]:
G=nx.from_pandas_dataframe(new2, 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', ['AIRLINE', 'DISTANCE','AIR_TIME','FLIGHT_NUMBER','ORIGIN_LAT','ORIGIN_LON'])

In [93]:
from bokeh.sampledata import us_states
from bokeh.plotting import *
us_states = us_states.data.copy()
del us_states['AK']
state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]
TOOLS = "pan,wheel_zoom,reset,hover,save"

s = figure(title="Graph Layout Demonstration", x_range=(-150,-60), 
           y_range=(20,60),toolbar_location="left",
              tools=TOOLS)

s.patches(state_xs, state_ys, fill_alpha=0.0,
    line_color="#884444", line_width=1.5)

GlyphRenderer(id='55391b1a-f836-4837-8835-3eb881cc6b19', ...)

In [94]:
s.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool())

In [95]:
source = ColumnDataSource(
    data=dict(
        lon=new2['DESTINATION_LON'],
        lat=new2['DESTINATION_LAT'],
    )
)

In [96]:
circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
s.add_glyph(source, circle)

GlyphRenderer(id='d0bc2c02-e7a2-4b7f-81d5-bcd8b8d9f0f7', ...)

In [97]:
show(s)

In [34]:
from bokeh.models import GraphRenderer, Oval
from bokeh.palettes import Spectral8, Viridis256
from bokeh.io import show, output_file

In [98]:
N = len(new2)
node_indices = list(range(N))

"""plot = figure(title="Graph Layout Demonstration", x_range=(-100,-50), y_range=(-60,20),
              tools="", toolbar_location=None)"""

graph = GraphRenderer()

graph.node_renderer.glyph = Oval(height=0.1, width=0.2, fill_color="fill_color")
graph.node_renderer.data_source.data = dict(
    index=node_indices,
    fill_color=Spectral8)

graph.edge_renderer.data_source.data = dict(
    start=[0]*N,
    end=node_indices)

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/bokeh/models/sources.py:138: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('fill_color', 8), ('index', 81)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


In [99]:
lon=new2['DESTINATION_LON']
lat=new2['DESTINATION_LAT']

In [100]:
graph_layout = dict(zip(node_indices, zip(lon, lat)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

In [101]:
s.renderers.append(graph)

In [102]:
show(s)

In [3]:
flights.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

In [4]:
flights.head(3)

YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE      ...        \
0            ANC                 SEA                    5      ...         
1            LAX                 PBI                   10      ...         
2            SFO                 CLT                   20      ...         

   ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  CANCELLATION_REASON  \
0         408.0          -22.0         0          0                  NaN   
1         741.0           -9.0         0          0                  NaN   
2         811.0            5.0         0          0                  NaN   

   AIR_SYSTEM_DELAY  SECURITY_DELAY  AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  \
0               NaN             NaN            NaN                  NaN   
1               NaN             NaN            NaN                  NaN   
2               NaN             NaN            NaN                  NaN   

   WEATHER_DELAY  
0            NaN  
1            NaN  
2            NaN  

[3 rows x 31 columns]